<a href="https://colab.research.google.com/github/EkeminiUmanah/g05-used-cars/blob/EkeminiUmanah-patch-2/UsedVehiclesPrediction2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Hamoye Stage G Used_Vehicles/vehiclesdf_WithUnknownCat.csv')
#In this dataset, missing values categorical features were filled with node, then the remaining missing values were group as a new category, uknown

In [6]:
df.isnull().sum()

price           0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
type            0
paint_color     0
car_age         0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190161 entries, 0 to 190160
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         190161 non-null  int64  
 1   manufacturer  190161 non-null  object 
 2   model         190161 non-null  object 
 3   condition     190161 non-null  object 
 4   cylinders     190161 non-null  object 
 5   fuel          190161 non-null  object 
 6   odometer      190161 non-null  float64
 7   title_status  190161 non-null  object 
 8   transmission  190161 non-null  object 
 9   drive         190161 non-null  object 
 10  type          190161 non-null  object 
 11  paint_color   190161 non-null  object 
 12  car_age       190161 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 18.9+ MB


In [8]:
df.corr()

,price,odometer,car_age
price,1.000000,-0.549779,-0.598009
odometer,-0.549779,1.000000,0.598766
car_age,-0.598009,0.598766,1.000000


In [9]:
from sklearn.preprocessing import LabelEncoder 

In [10]:
le = LabelEncoder()

In [ ]:
df.columns

Index(['price', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'title_status', 'transmission', 'drive', 'type',
       'paint_color', 'car_age'],
      dtype='object')

In [11]:
encoded_data = pd.get_dummies(df[['fuel', 'transmission', 'drive', 'type', 'paint_color']], drop_first=True)
encoded_data.head()

,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,transmission_manual,transmission_other,drive_fwd,drive_rwd,drive_unknown,type_bus,type_convertible,type_coupe,type_hatchback,type_mini-van,type_offroad,type_other,type_pickup,type_sedan,type_truck,type_unknown,type_van,type_wagon,paint_color_blue,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_unknown,paint_color_white,paint_color_yellow
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [12]:
encoded_data = pd.concat([df[['price', 'odometer', 'car_age', 'manufacturer', 'model', 'cylinders', 'condition', 'title_status']].apply(le.fit_transform), encoded_data], axis = 1)
encoded_data.head()

,price,odometer,car_age,manufacturer,model,cylinders,condition,title_status,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,transmission_manual,transmission_other,drive_fwd,drive_rwd,drive_unknown,type_bus,type_convertible,type_coupe,type_hatchback,type_mini-van,type_offroad,type_other,type_pickup,type_sedan,type_truck,type_unknown,type_van,type_wagon,paint_color_blue,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_unknown,paint_color_white,paint_color_yellow
0,1392,65745,23,13,13436,6,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,5481,75644,16,13,16110,6,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,4135,70058,8,12,9063,5,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,1744,44505,12,7,8441,3,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2122,68123,9,7,18051,5,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = encoded_data.drop(['price'], axis = 1)
X = scaler.fit_transform(X)

y = encoded_data['price']

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [15]:
!pip install catboost

     |████████████████████████████████| 66.3MB 51kB/s 


In [16]:
# import required algorithms
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor

# import evaluation metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# create a dictionary of the algorithms to be used:
models = {'KNeighborsRegressor':KNeighborsRegressor(),
        'RandomForestRegressor':RandomForestRegressor(),
        'ExtraTreesRegressor':ExtraTreesRegressor(),
        'GradientBoostingRegressor':GradientBoostingRegressor(),
        'DecisionTreeRegressor':DecisionTreeRegressor(),
        'CatBoostRegressor': CatBoostRegressor()}


In [17]:
#Funtion to compute RMSE, MAE, MSE, R2 scores

def pipeline(models, X_train, X_test, y_train, y_test):
    
    # store properties of each model
    prop = []
    
    # loop through the dictionary of models
    for reg_name, regressor in models.items():
        # empty dict for storing properties of each models
        reg_dict = {}
        # store the name of each model
        reg_dict['Name'] = reg_name
        # fit the regressor model
        regressor.fit(X_train, y_train)
        # compute r squared score
        reg_dict['r2'] = round(r2_score(y_test, regressor.predict(X_test)), 3)
        # compute the mean_absolute_error
        reg_dict['MAE'] = round(mean_absolute_error(y_test, regressor.predict(X_test)), 3)
        #compute the mean_squared_error
        reg_dict['MSE'] = round(mean_squared_error(y_test, regressor.predict(X_test)), 3)
        # compute the RMSE
        reg_dict['RMSE'] = round(np.sqrt(mean_squared_error(y_test, regressor.predict(X_test))), 3)
        # append the properties of a single regressor to the prop list
        prop.append(reg_dict)
     
    # create a dataframe with a list of all the model properties
    summary_df = pd.DataFrame(prop)
   
        
        
    return summary_df
        

In [18]:
import gc
gc.collect()

4

In [19]:
# execute the function
pipeline(models, X_train, X_test, y_train, y_test)

Learning rate set to 0.094372
0:	learn: 2821.9192433	total: 70.2ms	remaining: 1m 10s
1:	learn: 2677.0547039	total: 90ms	remaining: 44.9s
2:	learn: 2554.4286307	total: 109ms	remaining: 36.3s
3:	learn: 2443.1744258	total: 129ms	remaining: 32.1s
4:	learn: 2347.9631340	total: 147ms	remaining: 29.3s
5:	learn: 2264.6427099	total: 167ms	remaining: 27.7s
6:	learn: 2187.9765235	total: 194ms	remaining: 27.5s
7:	learn: 2124.2931777	total: 225ms	remaining: 27.8s
8:	learn: 2067.1223964	total: 243ms	remaining: 26.8s
9:	learn: 2015.0589329	total: 262ms	remaining: 25.9s
10:	learn: 1972.0779953	total: 280ms	remaining: 25.2s
11:	learn: 1931.9252857	total: 299ms	remaining: 24.6s
12:	learn: 1898.6597854	total: 318ms	remaining: 24.1s
13:	learn: 1867.3394789	total: 337ms	remaining: 23.8s
14:	learn: 1839.6479634	total: 356ms	remaining: 23.4s
15:	learn: 1815.6118120	total: 374ms	remaining: 23s
16:	learn: 1794.0387774	total: 393ms	remaining: 22.7s
17:	learn: 1774.0591527	total: 415ms	remaining: 22.6s
18:	learn

,Name,r2,MAE,MSE,RMSE
0,KNeighborsRegressor,0.735,1038.446,2368636.587,1539.038
1,RandomForestRegressor,0.854,722.826,1306005.001,1142.806
2,ExtraTreesRegressor,0.834,757.392,1479356.155,1216.288
3,GradientBoostingRegressor,0.739,1094.131,2329460.659,1526.257
4,DecisionTreeRegressor,0.731,924.967,2396435.112,1548.042
5,CatBoostRegressor,0.851,773.415,1332938.474,1154.530


In [23]:
import gc
gc.collect()

0

##### Hyperparameter tuning for the best performing model(Random Forest Regressor):

In [24]:
from sklearn.model_selection import GridSearchCV

n_estimators = [10, 50, 100, 200, 30]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]

params = {'n_estimators': n_estimators,
              'min_samples_leaf': min_samples_leaf,
              'min_samples_split': min_samples_split,
              'max_features': max_features}
rf_model = RandomForestRegressor()
grid_search = GridSearchCV(rf_model, params, cv=2, n_jobs=-1)

In [25]:
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(cv=2, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [26]:
# get best params
grid_search.best_params_

{'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 200}

In [27]:
# get best model
model = grid_search.best_estimator_

In [28]:
print(f'R2_score on training_set with tuned parameters: {model.score(X_train, y_train)}')
print(f'R2_score on testing_set with tuned parameters: {round(model.score(X_test, y_test), 3)}')
print(f'MSE_score on testing_set with tuned parameters: {round(mean_squared_error(y_test, model.predict(X_test)), 3)}')
print(f'RMSE_score on testing_set with tuned parameters: {round(np.sqrt(mean_squared_error(y_test, model.predict(X_test))), 3)}')
print(f'MAE_score with tuned parameters: {round(mean_absolute_error(y_test, model.predict(X_test)), 3)}')

R2_score on training_set with tuned parameters: 0.9725547655700719
R2_score on testing_set with tuned parameters: 0.855
MSE_score on testing_set with tuned parameters: 1296249.125
RMSE_score on testing_set with tuned parameters: 1138.529
MAE_score with tuned parameters: 721.066
